In [ ]:
import os
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader, Subset  # [변경됨]: random_split 대신 Subset 사용
import json
import pandas as pd
import numpy as np  # [변경됨]: numpy 임포트

# [추가됨] 재현성을 위한 시드 설정
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 데이터 경로 설정
data_dir = r"C:\Users\user\OneDrive\Desktop\Resnet18-real\data\brand_data2"
processed_data_dir = r"C:\Users\user\OneDrive\Desktop\Resnet18-real\data\processed_data"
json_save_dir = r"C:\Users\user\OneDrive\Desktop\Resnet18-real\json"
csv_save_dir = r"C:\Users\user\OneDrive\Desktop\Resnet18-real\csv"
model_save_path = r"C:\Users\user\OneDrive\Desktop\Resnet18-real\modelnew\resnet18.pth"

# JSON, CSV 저장 폴더 생성
os.makedirs(json_save_dir, exist_ok=True)
os.makedirs(csv_save_dir, exist_ok=True)

# 모든 브랜드 폴더를 순회하면서 하위 제품 폴더 이동
for brand in os.listdir(data_dir):
    brand_path = os.path.join(data_dir, brand)
    if os.path.isdir(brand_path):
        for product in os.listdir(brand_path):
            product_path = os.path.join(brand_path, product)
            if os.path.isdir(product_path):
                new_product_path = os.path.join(processed_data_dir, product)
                os.makedirs(new_product_path, exist_ok=True)
                for image in os.listdir(product_path):
                    src = os.path.join(product_path, image)
                    dst = os.path.join(new_product_path, image)
                    shutil.copy(src, dst)

print("✅ 모든 제품 폴더를 'processed_data'로 이동 완료!")

# 이미지 변환 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 로드
dataset = datasets.ImageFolder(root=processed_data_dir, transform=transform)
num_classes = len(dataset.classes)
print(f"총 클래스 개수: {num_classes}")
print("클래스 목록:", dataset.classes)

# [변경됨] 데이터셋 분할 - 클래스별로 7:2:1 비율로 분할
# 각 클래스별 인덱스 추출
class_indices = {cls: [] for cls in range(num_classes)}
for idx, (path, label) in enumerate(dataset.imgs):
    class_indices[label].append(idx)

train_indices, val_indices, test_indices = [], [], []
for label, indices in class_indices.items():
    np.random.shuffle(indices)  # 인덱스 랜덤 셔플
    n = len(indices)
    n_train = int(0.7 * n)
    n_val = int(0.2 * n)
    # 나머지는 테스트셋에 할당
    train_indices.extend(indices[:n_train])
    val_indices.extend(indices[n_train:n_train+n_val])
    test_indices.extend(indices[n_train+n_val:])

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)
# [변경됨 끝]

# JSON 파일로 데이터셋 저장
def save_json(subset_dataset, dataset_name):
    data_list = [{"image_path": subset_dataset.dataset.imgs[i][0], "label": int(subset_dataset.dataset.imgs[i][1])} 
                for i in subset_dataset.indices]
    json_path = os.path.join(json_save_dir, f"{dataset_name}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(data_list, f, indent=4)
    print(f"✅ {dataset_name}.json 저장 완료!")

save_json(train_dataset, "train")
save_json(val_dataset, "val")
save_json(test_dataset, "test")

# 데이터 로더 설정
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

print("✅ 데이터 로더 생성 완료!")

# 모델 설정 (ResNet18)
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# 손실 함수 및 최적화 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=1e-4)

# [추가됨] Learning rate scheduler (ReduceLROnPlateau 사용 - 검증 정확도를 기준으로 학습률 조절)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, verbose=True)

# 학습 기록 저장용 DataFrame
history = {"epoch": [], "train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

# [추가됨] Early Stopping 설정 변수 (개선이 없을 시 조기 종료)
early_stop_patience = 20
no_improve_count = 0

# [추가됨] 로그 파일 생성 (학습 기록 추가 저장)
log_file = os.path.join(csv_save_dir, "train_log.txt")
with open(log_file, "w") as lf:
    lf.write("Epoch,Train Loss,Train Acc,Val Loss,Val Acc,Learning Rate\n")

# 모델 학습 함수
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    global no_improve_count  # [추가됨] early stopping 카운트 변수 사용
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        train_loss, correct_train, total_train = 0.0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            correct_train += predicted.eq(labels).sum().item()
            total_train += labels.size(0)

        train_acc = 100 * correct_train / total_train
        train_loss /= len(train_loader)

        model.eval()
        val_loss, correct_val, total_val = 0.0, 0, 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                correct_val += predicted.eq(labels).sum().item()
                total_val += labels.size(0)

        val_acc = 100 * correct_val / total_val
        val_loss /= len(val_loader)

        history["epoch"].append(epoch + 1)
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(f"🚀 Epoch [{epoch+1}/{num_epochs}] | 📉 Train Loss: {train_loss:.4f} | 🎯 Train Accuracy: {train_acc:.4f} | 📑 Valid Loss: {val_loss:.4f} | 🎯 Valid Accuracy: {val_acc:.4f}")

        # [추가됨] Learning rate scheduler step (검증 정확도를 기준으로 학습률 조정)
        scheduler.step(val_acc)
        current_lr = optimizer.param_groups[0]['lr']
        print(f"현재 학습률: {current_lr:.6f}")

        # [추가됨] 로그 파일에 학습 기록 추가 저장
        with open(log_file, "a") as lf:
            lf.write(f"{epoch+1},{train_loss:.4f},{train_acc:.4f},{val_loss:.4f},{val_acc:.4f},{current_lr:.6f}\n")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), model_save_path)
            print(f"📌 새로운 최고 모델 저장됨! (Epoch {epoch+1}, Accuracy: {val_acc:.4f}) ✅")
            no_improve_count = 0  # [추가됨] 개선되었으므로 카운트 초기화
        else:
            no_improve_count += 1  # [추가됨] 개선되지 않음

        # [추가됨] Early Stopping 조건 확인 (개선이 없으면 조기 종료)
        if no_improve_count >= early_stop_patience:
            print(f"🚨 조기 종료: {early_stop_patience} 에폭 동안 개선 없음.")
            break

    print(f"🎯 최종 최고 검증 정확도: {best_acc:.2f}%")

# 모델 학습 실행
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=500)

# 학습 과정 CSV로 저장
df = pd.DataFrame(history)
csv_path = os.path.join(csv_save_dir, "텐트분류.csv")
df.to_csv(csv_path, index=False)
print(f"✅ 학습 기록 저장 완료: {csv_path}")

# 최적 모델 로드 후 테스트 데이터 평가
model.load_state_dict(torch.load(model_save_path))
model.eval()
correct_test, total_test = 0, 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        correct_test += predicted.eq(labels).sum().item()
        total_test += labels.size(0)

test_acc = 100 * correct_test / total_test
print(f"🎯 테스트 데이터셋 정확도: {test_acc:.2f}%")


Using device: cuda
✅ 모든 제품 폴더를 'processed_data'로 이동 완료!
총 클래스 개수: 59
클래스 목록: ['12X', '12Y', '4S 와이드', '6.3', 'A5', 'A7', 'IE', 'P1', 'X5', '가마보코', '날로', '노나돔', '니악', '도크돔', '라이더스', '랜드록', '랜드브리즈', '레라', '레이사', '리빙쉘', '문라이트', '미라클패밀리', '바랑에르돔', '발할', '뱅가드', '브이타프', '비무르', '비바돔', '빌리지', '새턴 쉘터', '새턴2룸', '솔로', '쉘터G', '스텔라릿지', '아고라', '아스가르드', '아퀼라', '아틀라스', '아티카', '알베르게', '알파인돔', '알페임', '어메니티돔', '오토듀얼팔레스', '와가야노', '우나', '우트가르드', '웨더마스터', '인디아나', '인스턴트업 3p', '캥거루', '코트텐트', '크로노스', '클라우드업', '투어링돔', '파프리카', '패스빅', '퍼시픽오션', '필드터널']
✅ train.json 저장 완료!
✅ val.json 저장 완료!
✅ test.json 저장 완료!
✅ 데이터 로더 생성 완료!


c:\Users\user\anaconda3\envs\classification\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


🚀 Epoch [1/500] | 📉 Train Loss: 2.1164 | 🎯 Train Accuracy: 51.9147 | 📑 Valid Loss: 1.1450 | 🎯 Valid Accuracy: 72.3174
현재 학습률: 0.000100
📌 새로운 최고 모델 저장됨! (Epoch 1, Accuracy: 72.3174) ✅
🚀 Epoch [2/500] | 📉 Train Loss: 0.8882 | 🎯 Train Accuracy: 79.4797 | 📑 Valid Loss: 0.8227 | 🎯 Valid Accuracy: 79.3379
현재 학습률: 0.000100
📌 새로운 최고 모델 저장됨! (Epoch 2, Accuracy: 79.3379) ✅
🚀 Epoch [3/500] | 📉 Train Loss: 0.5939 | 🎯 Train Accuracy: 85.3773 | 📑 Valid Loss: 0.7249 | 🎯 Valid Accuracy: 81.7922
현재 학습률: 0.000100
📌 새로운 최고 모델 저장됨! (Epoch 3, Accuracy: 81.7922) ✅
🚀 Epoch [4/500] | 📉 Train Loss: 0.4263 | 🎯 Train Accuracy: 88.9320 | 📑 Valid Loss: 0.7047 | 🎯 Valid Accuracy: 80.9932
현재 학습률: 0.000100
🚀 Epoch [5/500] | 📉 Train Loss: 0.3387 | 🎯 Train Accuracy: 90.7093 | 📑 Valid Loss: 0.6848 | 🎯 Valid Accuracy: 81.5068
현재 학습률: 0.000100
🚀 Epoch [6/500] | 📉 Train Loss: 0.3023 | 🎯 Train Accuracy: 91.2910 | 📑 Valid Loss: 0.6855 | 🎯 Valid Accuracy: 81.1073
현재 학습률: 0.000100
🚀 Epoch [7/500] | 📉 Train Loss: 0.2674 | 🎯 Tra

In [2]:
import os
import json
import torch
import torchvision.transforms as transforms
from PIL import Image
import torchvision.models as models
import torch.nn as nn
import torchvision.datasets as datasets

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 경로 설정
processed_data_dir = r"C:\Users\user\OneDrive\Desktop\Resnet18-real\data\processed_data"
json_save_dir = r"C:\Users\user\OneDrive\Desktop\Resnet18-real\json"
model_save_path = r"C:\Users\user\OneDrive\Desktop\Resnet18-real\modelnew\resnet18.pth"

# 평가용 이미지 전처리 (augmentation은 제거)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ImageFolder를 통해 클래스 정보를 불러옴
dummy_dataset = datasets.ImageFolder(root=processed_data_dir, transform=transform)
num_classes = len(dummy_dataset.classes)
class_names = dummy_dataset.classes
print(f"총 클래스 개수: {num_classes}")
print("클래스 목록:", class_names)

# 모델 구성 (ResNet18) 및 가중치 로드
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load(model_save_path, map_location=device))
model = model.to(device)
model.eval()
print("모델 로드 완료!")

def evaluate_json_dataset(json_path, dataset_name):
    """
    주어진 json 파일을 불러와서 각 이미지에 대해 예측 수행합니다.
    이미지별 정답/오답 여부(예측값과 정답)를 출력하고,
    마지막에 전체 정확도 및 클래스별 정확도를 출력합니다.
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data_list = json.load(f)
    
    total = len(data_list)
    correct = 0
    
    # 클래스별 정답/전체 카운터 초기화
    correct_by_class = {i: 0 for i in range(num_classes)}
    total_by_class = {i: 0 for i in range(num_classes)}
    
    print(f"\n===== {dataset_name.upper()} 데이터셋 평가 시작 (총 {total}개 이미지) =====")
    
    for sample in data_list:
        image_path = sample["image_path"]
        true_label = sample["label"]
        total_by_class[true_label] += 1
        
        # 이미지 로드 및 전처리
        try:
            image = Image.open(image_path).convert("RGB")
        except Exception as e:
            print(f"이미지 로드 실패: {image_path} ({e})")
            continue
        input_tensor = transform(image).unsqueeze(0).to(device)
        
        # 예측 수행
        with torch.no_grad():
            output = model(input_tensor)
            _, pred_label = torch.max(output, 1)
            pred_label = pred_label.item()
        
        # 정답/오답 여부 판단
        if pred_label == true_label:
            correct += 1
            correct_by_class[true_label] += 1
            result = "✅ 정답"
        else:
            result = f"❌ 오답 (예측: {pred_label}, 정답: {true_label})"
        
        print(f"{os.path.basename(image_path)} -> {result}")
    
    overall_acc = 100 * correct / total if total > 0 else 0
    print(f"\n===== {dataset_name.upper()} 데이터셋 전체 정확도: {correct} / {total} = {overall_acc:.2f}% =====")
    
    print(f"----- {dataset_name.upper()} 데이터셋 클래스별 정확도 -----")
    for cls in range(num_classes):
        cls_total = total_by_class[cls]
        cls_correct = correct_by_class[cls]
        if cls_total > 0:
            cls_acc = 100 * cls_correct / cls_total
        else:
            cls_acc = 0
        print(f"[{class_names[cls]}] : {cls_correct} / {cls_total} = {cls_acc:.2f}%")
    
    print("======================================================\n")
    
    # 전체 정확도, 클래스별 정답/전체를 반환
    return overall_acc, correct_by_class, total_by_class

# JSON 파일 경로 설정 및 평가 실행
results = {}
for ds in ["train", "val", "test"]:
    json_path = os.path.join(json_save_dir, f"{ds}.json")
    if os.path.exists(json_path):
        print(f"== {ds.upper()} 데이터셋 평가 ==")
        # evaluate_json_dataset 함수가 (overall_acc, correct_by_class, total_by_class)를 반환
        results[ds] = evaluate_json_dataset(json_path, ds)
    else:
        print(f"{ds}.json 파일을 찾을 수 없습니다.")

# ===== 최종 요약 출력 (train, val, test 정확도) =====
print("===== 전체 요약 =====")
for ds in ["train", "val", "test"]:
    if ds in results:
        overall_acc, _, _ = results[ds]
        print(f"{ds.upper()} 데이터셋 최종 정확도: {overall_acc:.2f}%")


Using device: cuda
총 클래스 개수: 59
클래스 목록: ['12X', '12Y', '4S 와이드', '6.3', 'A5', 'A7', 'IE', 'P1', 'X5', '가마보코', '날로', '노나돔', '니악', '도크돔', '라이더스', '랜드록', '랜드브리즈', '레라', '레이사', '리빙쉘', '문라이트', '미라클패밀리', '바랑에르돔', '발할', '뱅가드', '브이타프', '비무르', '비바돔', '빌리지', '새턴 쉘터', '새턴2룸', '솔로', '쉘터G', '스텔라릿지', '아고라', '아스가르드', '아퀼라', '아틀라스', '아티카', '알베르게', '알파인돔', '알페임', '어메니티돔', '오토듀얼팔레스', '와가야노', '우나', '우트가르드', '웨더마스터', '인디아나', '인스턴트업 3p', '캥거루', '코트텐트', '크로노스', '클라우드업', '투어링돔', '파프리카', '패스빅', '퍼시픽오션', '필드터널']
모델 로드 완료!
== TRAIN 데이터셋 평가 ==

===== TRAIN 데이터셋 평가 시작 (총 6189개 이미지) =====
D8SrJ3kvTV2u4SPjU0dibw.jpeg -> ✅ 정답
naver_네이처하이크 12X 텐트_57.jpg -> ✅ 정답
google_NatureHike 12X tent_45.jpg -> ✅ 정답
1663482870117000ccq_L7Cxg.jpg -> ✅ 정답
google_네이처하이크 12X 텐트_19.jpg -> ❌ 오답 (예측: 1, 정답: 0)
naver_네이처하이크 12X 텐트_92.jpg -> ✅ 정답
google_NatureHike 12X tent_24_1.jpg -> ✅ 정답
google_네이처하이크 12X 텐트_6_1.jpg -> ✅ 정답
google_NatureHike 12X tent_28.jpg -> ✅ 정답
naver_네이처하이크 12X 텐트_2_2.jpg -> ✅ 정답
naver_네이처하이크 12X 텐트_69.jpg -> ✅ 정답
go